In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import os
import math
from ast import literal_eval
from collections import defaultdict
from matplotlib_venn import venn3, venn3_circles
from matplotlib_venn import venn2, venn2_circles


# Datasets

## Genres and Platforms

In [ ]:
df_genres = pd.read_csv('../genres.csv', header=None)
df_genres.columns = ['genre_id', 'genre_name']

df_platforms = pd.read_csv('../platforms.csv', header=None)
df_platforms.columns = ['platform_id', 'platform_name']

list_platform_ps = ['PlayStation 4', 'PlayStation 3', 'PlayStation 2', 'PlayStation', 'PlayStation Vita', 'PlayStation Portable']
list_platform_xbox = ['Xbox One', 'Xbox 360', 'Xbox', 'Xbox Live Arcade']
list_platform_pc = ['PC (Microsoft Windows)', 'PC DOS', 'Mac', 'Linux']

# show genres columns
df_genres.head()

## HLTB

In [ ]:
if not os.path.exists('dataset_hltb.csv'):

    PATH = '../igdb_scarper/hl2b/'

    dataset_hltb = pd.DataFrame()
        
    for file in os.listdir(PATH):

        try:
            df = pd.read_csv(PATH + '/' + file, lineterminator='\n', encoding="utf-8",)
            dataset_hltb = pd.concat([dataset_hltb, df])
        except pd.errors.EmptyDataError:
            print(file, " is empty and has been skipped.")    
            # pass

    dataset_hltb = dataset_hltb.drop(columns=['Unnamed: 0'])    
    dataset_hltb = dataset_hltb.drop_duplicates(subset=['name'], keep='first')

    dataset_hltb.to_csv('dataset_hltb.csv', index=False, encoding='utf-8', lineterminator='\n')

    dataset_hltb.info()


## IGDB

In [ ]:
if not os.path.exists('dataset_igdb.csv'):

    PATH = '../igdb_scarper/IGDB_metadata/'

    dataset = pd.DataFrame()
    for platform in os.listdir(PATH):
        # print(platform)

        df_platform = pd.DataFrame()
        
        for file in os.listdir(PATH + platform):

            try:
                df_platform = pd.read_csv(PATH + platform + '/' + file, lineterminator='\n', encoding="utf-8")
                dataset = pd.concat([dataset, df_platform])
            except pd.errors.EmptyDataError:
                # print(file, " is empty and has been skipped.")    
                pass

    dataset['first_release_date'] = pd.to_datetime(dataset['first_release_date'], unit='s')

    # remove the duplicated ids
    dataset = dataset.drop_duplicates(subset=['id'], keep='first')
    
    dataset.to_csv('dataset_igdb.csv', index=False, encoding='utf-8', lineterminator='\n')

    dataset.info()

## IGDB + HLTB

In [ ]:
if not os.path.exists('dataset_igdb_hltb.csv'):

    dataset_hltb = pd.read_csv('dataset_hltb.csv', lineterminator='\n', encoding="utf-8")

    dataset = pd.read_csv('dataset_igdb.csv', lineterminator='\n', encoding="utf-8")

    # concatenate the two datasets
    dataset = pd.merge(dataset, dataset_hltb, how='left', left_on='name', right_on='name')

    dataset.to_csv('dataset_igdb_hltb.csv', index=False, encoding='utf-8', lineterminator='\n')

else:
    dataset = pd.read_csv('dataset_igdb_hltb.csv', lineterminator='\n', encoding="utf-8",
                            converters={"id": str, 
                                        "first_release_date": pd.to_datetime,
                                        "platforms": literal_eval,
                                        "genres": lambda x: x.strip("[]").split(", ") if x != 'Missing' else 'Missing'
                                        })

dataset.info()


In [ ]:
# Total number of games
print("Total number of games: ", dataset.shape[0])


In [ ]:
# show the missing values in the dataset
dataset.isnull().sum()

# Platform

## Games per platform

In [ ]:
# number of games per platform
games_per_platform = dataset['platforms'].apply(lambda x: x[0]).value_counts().sort_values(ascending=False)

games_per_platform = pd.DataFrame(games_per_platform).reset_index().rename(columns={'platforms': 'platform_id'})
games_per_platform['platform_id'] = games_per_platform['platform_id'].astype(int)
games_per_platform = games_per_platform.merge(df_platforms, on='platform_id', how='left')
games_per_platform.sort_values(by='count', ascending=False)

games_per_platform['count'].sum()
games_per_platform


## Single platform games (exclusives)

In [ ]:

# only single platform games
dataset[dataset['platforms'].apply(lambda x: len(x) == 1)]

# show only the single platform games in their respective platform
games_single_platform = dataset[dataset['platforms'].apply(lambda x: len(x) == 1)]['platforms'].apply(lambda x: x[0]).value_counts()

games_single_platform = pd.DataFrame(games_single_platform).reset_index().rename(columns={'platforms': 'platform_id'})
games_single_platform['platform_id'] = games_single_platform['platform_id'].astype(int)
games_single_platform = games_single_platform.merge(df_platforms, on='platform_id', how='left')
games_single_platform.sort_values(by='count', ascending=False)

# total number of single platform games
print("Total number of single platform games: ", games_single_platform['count'].sum())


# Genres

## Games per genre

In [ ]:
# remove values missing values in a column
dataset['genres'].replace('Missing', np.nan, inplace=True)
# replace '' with np.nan
dataset['genres'].replace('', np.nan, inplace=True)

print("Number of games with missing genres: ", dataset['genres'].isnull().sum())

dataset['genres'].sort_values(ascending=True)

In [ ]:
# remove genres values missing values
dataset_no_nas = dataset.dropna(subset=['genres'])
# remove lists with empty strings
dataset_no_nas = dataset_no_nas[dataset_no_nas['genres'].apply(lambda x: x[0] != '')]

games_per_genre = dataset_no_nas['genres'].apply(lambda x: x[0]).value_counts().sort_values(ascending=False)

games_per_genre = pd.DataFrame(games_per_genre).reset_index().rename(columns={'genres': 'genre_id'})
games_per_genre['genre_id'] = games_per_genre['genre_id'].astype(int)
games_per_genre = games_per_genre.merge(df_genres, on='genre_id', how='left')
games_per_genre.sort_values(by='count', ascending=False)

games_per_genre['count'].sum()
games_per_genre


## RPG venn

In [ ]:
# list all games that contains the genre '12' in the list of genres
dataset_rpg = dataset_no_nas[dataset_no_nas['genres'].apply(lambda x: '12' in x)]

# venn diagram with the genres in the dataset_rpg
games_per_genre_rpg = dataset_rpg['genres'].apply(lambda x: x[0]).value_counts().sort_values(ascending=False)

games_per_genre_rpg = pd.DataFrame(games_per_genre_rpg).reset_index().rename(columns={'genres': 'genre_id'})
games_per_genre_rpg['genre_id'] = games_per_genre_rpg['genre_id'].astype(int)
games_per_genre_rpg = games_per_genre_rpg.merge(df_genres, on='genre_id', how='left')
games_per_genre_rpg.sort_values(by='count', ascending=False)

# venn diagram
fig = plt.figure(figsize=(8, 8))

v = venn3(subsets=(games_per_genre_rpg['count'][0], 
               games_per_genre_rpg['count'][1], 
               games_per_genre_rpg['count'][2], 
               games_per_genre_rpg['count'][3], 
               games_per_genre_rpg['count'][4], 
               games_per_genre_rpg['count'][5], 
               games_per_genre_rpg['count'][6],
               games_per_genre_rpg['count'][7],
               games_per_genre_rpg['count'][8]))

c = venn3_circles(subsets=(games_per_genre_rpg['count'][0], 
               games_per_genre_rpg['count'][1], 
               games_per_genre_rpg['count'][2], 
               games_per_genre_rpg['count'][3], 
               games_per_genre_rpg['count'][4], 
               games_per_genre_rpg['count'][5], 
               games_per_genre_rpg['count'][6],
               games_per_genre_rpg['count'][7],
               games_per_genre_rpg['count'][8]))

# c[0].set_lw(1.0)
# c[0].set_ls('dotted')

# remove the labels
for text in v.set_labels:
     text.set_text("")

# plt.title("Venn diagram of the genres")

plt.legend(games_per_genre_rpg['genre_name'].tolist())

# plt.show()

games_per_genre_rpg


# Year

## Games per Year

In [ ]:

# games_per_year = dataset['first_release_date'].dt.year.value_counts().sort_index().reset_index()
# games_per_year.columns = ['year', 'count']
# games_per_year.sort_values(by='year', ascending=False)

# table with the number of games per year keep years type
dataset['first_release_date'].dt.year.value_counts()

# dataset.head()


In [ ]:
# plot the games per year
fig, ax = plt.subplots(figsize=(20, 10))
fig.tight_layout()
# fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.2)

# dataset = dataset[(dataset['first_release_date'].dt.year != 1992) & (dataset['first_release_date'].dt.year != 2024)]

# start_year = dataset['first_release_date'].dt.year.min()
# end_year = dataset['first_release_date'].dt.year.max()

# bin_edges = np.arange(start=start_year, stop=end_year + 1, step=1)

# Convert bin_edges back to datetime for plotting
# bin_edges = pd.to_datetime(bin_edges, format='%Y')

ax.hist(dataset['first_release_date'], 
        bins=len(dataset['first_release_date'].dt.year.unique()),
        color='lightgray', 
        edgecolor='black',
        align='left',
        linewidth=1.2)

# add number of games on top of each bar
for i in ax.patches:
    ax.text(i.get_x() + i.get_width() / 2, i.get_height() + 100, str(int(i.get_height())), 
            fontsize=16, rotation=90, ha='center', va='bottom', color='black')

# add a second y axis with the average main value per year from main_per_year
main_per_year = dataset[(dataset['main'] != 0)].groupby(
    dataset['first_release_date'].dt.year)['main'].mean().reset_index().rename(columns={'main': 'mean_main'})
# add the completionist time per year
main_per_year['mean_completionist'] = dataset[(dataset['completionist'] != 0)].groupby(
    dataset['first_release_date'].dt.year)['completionist'].mean().reset_index()['completionist']
# add the main+extra time per year
main_per_year['mean_main_extra'] = dataset[(dataset['extra'] != 0)].groupby(
    dataset['first_release_date'].dt.year)['extra'].mean().reset_index()['extra']
# transform the year to datetime
main_per_year['first_release_date'] = pd.to_datetime(main_per_year['first_release_date'], format='%Y')

ax2 = ax.twinx()
ax2.plot(main_per_year['first_release_date'], main_per_year['mean_main'], 
         color='red', linewidth=3, linestyle='--', label='main', alpha=0.5, marker='o', markersize=10, markerfacecolor='red', markeredgecolor='red', markeredgewidth=2)
ax2.plot(main_per_year['first_release_date'], main_per_year['mean_completionist'], 
         color='blue', linewidth=3, linestyle='--', label='completionist', alpha=0.5, marker='o', markersize=10, markerfacecolor='blue', markeredgecolor='blue', markeredgewidth=2)
ax2.plot(main_per_year['first_release_date'], main_per_year['mean_main_extra'], 
         color='green', linewidth=3, linestyle='--', label='main+extra', alpha=0.5, marker='o', markersize=10, markerfacecolor='green', markeredgecolor='green', markeredgewidth=2)
ax2.set_ylabel('Average time to complete the game (hours)')

# add legend
# ax.legend(loc='upper left')
ax2.legend(loc='upper left', fontsize=16)

# increase the size of the x and y axis labels
ax.tick_params(axis='both', which='major', labelsize=16)
ax2.tick_params(axis='both', which='major', labelsize=16)

# increase the font size of the x and y axis labels
ax.xaxis.label.set_size(18)
ax.yaxis.label.set_size(18)
ax2.yaxis.label.set_size(18)

ax.set_xlabel('Year')
ax.set_ylabel('Number of games')
# ax.set_title('Number of games per year')

# # add horizontal grid lines
ax.grid(axis='y', alpha=0.75)

ax.xaxis.set_major_locator(dates.YearLocator())
ax.xaxis.set_major_formatter(dates.DateFormatter('%y'))

# plt.style.available
fig.savefig('hist.pdf', bbox_inches="tight", dpi=300, format='pdf')
